# SQLAlchemy

## Instlando paquetes

In [1]:
pip install SQLAlchemy

Note: you may need to restart the kernel to use updated packages.


## Importacion dee paquete necesarios

In [1]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, DateTime
from sqlalchemy.orm import sessionmaker, relationship, declarative_base
from datetime import datetime


# Configuracion basica

In [2]:
# 1. CONFIGURACIÓN
# Crear la clase base para los modelos
Base = declarative_base()

# Crear el motor de la base de datos
# sqlite:///tareas.db -> Crea un archivo tareas.db en el directorio actual
# echo=True -> Muestra las consultas SQL en la consola (útil para debugging)
engine = create_engine('sqlite:///tareas.db', echo=True)


## Definicion de modelos

In [3]:
# 2. DEFINICIÓN DE MODELOS
class Usuario(Base):
    """Modelo para la tabla de usuarios"""
    __tablename__ = 'usuarios'  # Nombre de la tabla en la base de datos
    
    # Definición de columnas
    id = Column(Integer, primary_key=True)  # Identificador único
    nombre = Column(String(50), nullable=False)  # Nombre del usuario, no puede ser nulo
    email = Column(String(50), unique=True, nullable=False)  # Email único del usuario
    
    # Relación uno a muchos con la tabla Tareas
    # back_populates permite acceder a las tareas desde el usuario
    tareas = relationship("Tarea", back_populates="usuario")

    def __repr__(self):
        """Representación en string del objeto Usuario"""
        return f"<Usuario(nombre='{self.nombre}', email='{self.email}')>"

In [4]:
class Tarea(Base):
    """Modelo para la tabla de tareas"""
    __tablename__ = 'tareas'
    
    # Definición de columnas
    id = Column(Integer, primary_key=True)
    titulo = Column(String(100), nullable=False)
    descripcion = Column(String(200))
    fecha_creacion = Column(DateTime, default=datetime.utcnow)  # Fecha automática al crear
    
    # Clave foránea que relaciona con la tabla usuarios
    usuario_id = Column(Integer, ForeignKey('usuarios.id'))
    
    # Relación muchos a uno con la tabla Usuarios
    usuario = relationship("Usuario", back_populates="tareas")

    def __repr__(self):
        """Representación en string del objeto Tarea"""
        return f"<Tarea(titulo='{self.titulo}', usuario_id={self.usuario_id})>"


## Crear las tablas definidas en el modelo

In [5]:
# Crear todas las tablas en la base de datos
Base.metadata.create_all(engine)

2024-12-13 15:06:31,393 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-13 15:06:31,395 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("usuarios")
2024-12-13 15:06:31,397 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-13 15:06:31,399 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("tareas")
2024-12-13 15:06:31,400 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-13 15:06:31,403 INFO sqlalchemy.engine.Engine COMMIT


## Sessions

In [6]:
# Crear una fábrica de sesiones y una sesión
Session = sessionmaker(bind=engine)
session = Session()

## Metodos CRUD

In [11]:
usuario_test = Usuario(nombre='Juan', email='ja@gmail.com')

In [16]:
usuario_test

<Usuario(nombre='pedro', email='ja@gmail.com')>

In [7]:
def crear_usuario(nombre, email):
    """
    Crea un nuevo usuario en la base de datos.
    Args:
        nombre (str): Nombre del usuario
        email (str): Email del usuario
    Returns:
        Usuario: Objeto usuario creado
    """
    usuario = Usuario(nombre=nombre, email=email)
    session.add(usuario)  # Agregar a la sesión
    session.commit()  # Guardar cambios en la base de datos
    return usuario

def crear_tarea(titulo, descripcion, usuario_id):
    """
    Crea una nueva tarea para un usuario específico.
    Args:
        titulo (str): Título de la tarea
        descripcion (str): Descripción detallada de la tarea
        usuario_id (int): ID del usuario al que se asigna la tarea
    Returns:
        Tarea: Objeto tarea creado
    """
    tarea = Tarea(titulo=titulo, descripcion=descripcion, usuario_id=usuario_id)
    session.add(tarea)
    session.commit()
    return tarea

def obtener_usuarios():
    """
    Obtiene todos los usuarios de la base de datos.
    Returns:
        list: Lista de objetos Usuario
    """
    return session.query(Usuario).all()

def obtener_tareas():
    """
    Obtiene todas las tareas de la base de datos.
    Returns:
        list: Lista de objetos Tarea
    """
    return session.query(Tarea).all()

def obtener_tareas_usuario(usuario_id):
    """
    Obtiene todas las tareas de un usuario específico.
    Args:
        usuario_id (int): ID del usuario
    Returns:
        list: Lista de tareas del usuario
    """
    return session.query(Tarea).filter_by(usuario_id=usuario_id).all()

def actualizar_tarea(tarea_id, nuevo_titulo=None, nueva_descripcion=None):
    """
    Actualiza una tarea existente.
    Args:
        tarea_id (int): ID de la tarea a actualizar
        nuevo_titulo (str, opcional): Nuevo título para la tarea
        nueva_descripcion (str, opcional): Nueva descripción para la tarea
    Returns:
        Tarea: Objeto tarea actualizado
    """
    tarea = session.query(Tarea).filter_by(id=tarea_id).first()
    if tarea:
        if nuevo_titulo:
            tarea.titulo = nuevo_titulo
        if nueva_descripcion:
            tarea.descripcion = nueva_descripcion
        session.commit()
    return tarea

def eliminar_tarea(tarea_id):
    """
    Elimina una tarea de la base de datos.
    Args:
        tarea_id (int): ID de la tarea a eliminar
    Returns:
        bool: True si se eliminó correctamente, False si no se encontró la tarea
    """
    tarea = session.query(Tarea).filter_by(id=tarea_id).first()
    if tarea:
        session.delete(tarea)
        session.commit()
        return True
    return False

In [8]:
crear_usuario(nombre='dannkol', email='dannkol@gmail.com')

2024-12-13 15:06:40,382 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-13 15:06:40,386 INFO sqlalchemy.engine.Engine INSERT INTO usuarios (nombre, email) VALUES (?, ?)
2024-12-13 15:06:40,389 INFO sqlalchemy.engine.Engine [generated in 0.00293s] ('dannkol', 'dannkol@gmail.com')
2024-12-13 15:06:40,392 INFO sqlalchemy.engine.Engine COMMIT
2024-12-13 15:06:40,405 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-13 15:06:40,408 INFO sqlalchemy.engine.Engine SELECT usuarios.id AS usuarios_id, usuarios.nombre AS usuarios_nombre, usuarios.email AS usuarios_email 
FROM usuarios 
WHERE usuarios.id = ?
2024-12-13 15:06:40,409 INFO sqlalchemy.engine.Engine [generated in 0.00108s] (1,)


<Usuario(nombre='dannkol', email='dannkol@gmail.com')>

In [9]:
user = obtener_usuarios()

2024-12-13 15:06:44,420 INFO sqlalchemy.engine.Engine SELECT usuarios.id AS usuarios_id, usuarios.nombre AS usuarios_nombre, usuarios.email AS usuarios_email 
FROM usuarios
2024-12-13 15:06:44,422 INFO sqlalchemy.engine.Engine [generated in 0.00265s] ()


In [10]:
user

[<Usuario(nombre='dannkol', email='dannkol@gmail.com')>]

In [11]:
user[0].nombre

'dannkol'

In [12]:
user[0].nombre = ' luis'

In [13]:
user

[<Usuario(nombre=' luis', email='dannkol@gmail.com')>]

In [14]:
session.commit()

2024-12-13 15:08:32,406 INFO sqlalchemy.engine.Engine UPDATE usuarios SET nombre=? WHERE usuarios.id = ?
2024-12-13 15:08:32,408 INFO sqlalchemy.engine.Engine [generated in 0.00322s] (' luis', 1)
2024-12-13 15:08:32,411 INFO sqlalchemy.engine.Engine COMMIT


In [15]:
obtener_usuarios()

2024-12-13 15:08:47,056 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-13 15:08:47,058 INFO sqlalchemy.engine.Engine SELECT usuarios.id AS usuarios_id, usuarios.nombre AS usuarios_nombre, usuarios.email AS usuarios_email 
FROM usuarios
2024-12-13 15:08:47,061 INFO sqlalchemy.engine.Engine [cached since 122.6s ago] ()


[<Usuario(nombre=' luis', email='dannkol@gmail.com')>]

In [13]:
task = crear_tarea(usuario_id=user[0].id, titulo='Tarea 1', descripcion='Descripcion de la tarea 1')

2024-12-13 13:44:56,278 INFO sqlalchemy.engine.Engine INSERT INTO tareas (titulo, descripcion, fecha_creacion, usuario_id) VALUES (?, ?, ?, ?)
2024-12-13 13:44:56,280 INFO sqlalchemy.engine.Engine [generated in 0.00189s] ('Tarea 1', 'Descripcion de la tarea 1', '2024-12-13 18:44:56.278756', 1)
2024-12-13 13:44:56,282 INFO sqlalchemy.engine.Engine COMMIT


In [14]:
task.descripcion

2024-12-13 13:44:56,295 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-13 13:44:56,297 INFO sqlalchemy.engine.Engine SELECT tareas.id AS tareas_id, tareas.titulo AS tareas_titulo, tareas.descripcion AS tareas_descripcion, tareas.fecha_creacion AS tareas_fecha_creacion, tareas.usuario_id AS tareas_usuario_id 
FROM tareas 
WHERE tareas.id = ?
2024-12-13 13:44:56,299 INFO sqlalchemy.engine.Engine [generated in 0.00132s] (1,)


'Descripcion de la tarea 1'

In [15]:
get_taks = obtener_tareas_usuario(usuario_id=user[0].id)

2024-12-13 13:44:56,312 INFO sqlalchemy.engine.Engine SELECT usuarios.id AS usuarios_id, usuarios.nombre AS usuarios_nombre, usuarios.email AS usuarios_email 
FROM usuarios 
WHERE usuarios.id = ?
2024-12-13 13:44:56,314 INFO sqlalchemy.engine.Engine [cached since 0.07503s ago] (1,)
2024-12-13 13:44:56,317 INFO sqlalchemy.engine.Engine SELECT tareas.id AS tareas_id, tareas.titulo AS tareas_titulo, tareas.descripcion AS tareas_descripcion, tareas.fecha_creacion AS tareas_fecha_creacion, tareas.usuario_id AS tareas_usuario_id 
FROM tareas 
WHERE tareas.usuario_id = ?
2024-12-13 13:44:56,318 INFO sqlalchemy.engine.Engine [generated in 0.00153s] (1,)


In [23]:
get_taks[0].id

1

In [24]:
eliminar_tarea(tarea_id=get_taks[0].id)

2024-12-13 13:48:10,943 INFO sqlalchemy.engine.Engine SELECT tareas.id AS tareas_id, tareas.titulo AS tareas_titulo, tareas.descripcion AS tareas_descripcion, tareas.fecha_creacion AS tareas_fecha_creacion, tareas.usuario_id AS tareas_usuario_id 
FROM tareas 
WHERE tareas.id = ?
 LIMIT ? OFFSET ?
2024-12-13 13:48:10,945 INFO sqlalchemy.engine.Engine [generated in 0.00302s] (1, 1, 0)
2024-12-13 13:48:10,949 INFO sqlalchemy.engine.Engine DELETE FROM tareas WHERE tareas.id = ?
2024-12-13 13:48:10,950 INFO sqlalchemy.engine.Engine [generated in 0.00132s] (1,)
2024-12-13 13:48:10,952 INFO sqlalchemy.engine.Engine COMMIT


True

In [25]:
obtener_tareas_usuario(usuario_id=user[0].id)

2024-12-13 13:48:20,475 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-13 13:48:20,477 INFO sqlalchemy.engine.Engine SELECT usuarios.id AS usuarios_id, usuarios.nombre AS usuarios_nombre, usuarios.email AS usuarios_email 
FROM usuarios 
WHERE usuarios.id = ?
2024-12-13 13:48:20,478 INFO sqlalchemy.engine.Engine [cached since 204.2s ago] (1,)
2024-12-13 13:48:20,480 INFO sqlalchemy.engine.Engine SELECT tareas.id AS tareas_id, tareas.titulo AS tareas_titulo, tareas.descripcion AS tareas_descripcion, tareas.fecha_creacion AS tareas_fecha_creacion, tareas.usuario_id AS tareas_usuario_id 
FROM tareas 
WHERE tareas.usuario_id = ?
2024-12-13 13:48:20,481 INFO sqlalchemy.engine.Engine [cached since 204.2s ago] (1,)


[]

In [19]:
try:
    usr1 = Usuario(nombre="Danvid", email="david@gmail.com")
    usr2 = Usuario(nombre="juan", email="juan@gmail.com", password="123456")
    
    session.add(usr1)
    session.add(usr2)
    session.commit()

except Exception as e:
    print(e)
    session.rollback()

'password' is an invalid keyword argument for Usuario
